In [2]:
import numpy as np
import pandas as pd

In [254]:
ed = pd.read_csv("../data/spam_or_not_spam.csv")
ed.iloc[[2]]
def parse_data(data):
        words = {}
        for e in range(len(data['email'])):
            e_ar = []
            if isinstance(data['email'][e], str):
                e_ar = data['email'][e].split()
            for w in e_ar:
                if w in words:
                    words[w] = words[w] + [data['label'][e]]
                else:
                    words[w] = [] + [data['label'][e]]
        
        return words

words = parse_data(ed)


In [281]:


class NaiveBayes:
    def __init__(self, MAX_WORDS=50000):
        self.words = {}
        self.n = 0
        self.max_words = MAX_WORDS
        self.px_y1 = {}
        self.px_y0 = {}
        self.y1_total = 0
        self.y0_total = 0
        self.p_y1 = 0
        self.p_y0 = 0
        
        
    def fit(self, emails, words):
        self.words = words
        self.n = len(self.words)
        self.cal_p_y(emails)
        self.calc_probs(words)
    
    
    
    def predict(self, email):
        words = self.parse_email(email)
        p_x_y1 = self.p_y1
        p_x_y0 = self.p_y0
        for w in words:
            if w in self.px_y1:
                p_x_y1 *= self.px_y1[w]
            if w in self.px_y0:
                p_x_y0 *= self.px_y0[w]
         
        
        if p_x_y1 == 0:
            p_x_y1 +=1
        if p_x_y0 == 0:
            p_x_y0 += 1
    
            
        p_y1 = p_x_y1 / (p_x_y1 + p_x_y0)
        p_y0 = p_x_y0 / (p_x_y1 + p_x_y0)    
        

        return 1 if p_y1 >= p_y0 else 0
    
            
        
    
    def cal_p_y(self, emails):
        for l in emails['label']:
            if l == 1:
                self.y1_total += 1
            else:
                self.y0_total += 1
        

        self.p_y1 = self.y1_total / self.n
        self.p_y0 = 1 - self.p_y1
        print(self.p_y1, self.p_y0)
        
        
    def calc_probs(self, words):
        for w in self.words:
            for label in self.words[w]:
                if label == 1:
                    if w not in self.px_y1:
                        self.px_y1[w] = 0
                    self.px_y1[w] +=1
                else:
                    if w not in self.px_y0:
                        self.px_y0[w] = 0
                    self.px_y0[w] +=1
        
            if w not in self.px_y1:
                self.px_y1[w] = 0
                self.px_y1[w] += 1

            if w not in self.px_y0:
                self.px_y0[w] = 0
                self.px_y0[w] += 1
        
            self.px_y1[w] /= (self.y1_total + 2)
            self.px_y0[w] /= (self.y0_total + 2)
            
        
        
    def parse_email(self, email):
        x = []
        if isinstance(email, str):
            for w in email.split():
                if w not in x:
                    x.append(w)
        return x
            
    

    


In [282]:
nb_classifier = NaiveBayes()
nb_classifier.fit(ed, words)

0.01461048448366548 0.9853895155163345


In [285]:
c = 0
for i in range(2999):
    if nb_classifier.predict(ed.iloc[i]['email']) == ed.iloc[i]['label']:
        c += 1

print(c)

2193
